In [1]:
import numpy as np                                          
from mindquantum.core.circuit import Circuit                
from mindquantum.core.gates import H, RX, RY, RZ,X    
from mindquantum.core.parameterresolver import PRGenerator  
from mindquantum.simulator import Simulator
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split   
from mindquantum.algorithm.library import amplitude_encoder
from mindquantum.algorithm.nisq import IQPEncoding
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
from mindquantum.core.operators import QubitOperator           # 导入QubitOperator模块，用于构造泡利算符
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
import mindspore as ms                                                                         # 导入mindspore库并简写为ms
from mindquantum.framework import MQLayer,MQN2Layer,MQOps                                              # 导入MQLayer
# 导入HardwareEfficientAnsatz
from mindquantum.core.gates import RY           
from mindquantum.core.circuit import UN
import torch
from torchvision import datasets, transforms# 导入量子门RY
from scipy.ndimage import zoom
import random
from data_processing import PCA_data_preprocessing

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [3]:
# # 下载和加载 MNIST 数据集
# mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
# X_train, X_test, y_train, y_test = PCA(mnist_dataset,8)

In [3]:

# encoder.summary()                                    # 总结Encoprg = PRGenerator('alpha')
nqbits = 8
encoder = Circuit()
encoder += UN(H, nqbits)                                  # H门作用在每1位量子比特
for i in range(nqbits):                                   # i = 0, 1, 2, 3
    encoder += RY(prg.new()).on(i)                 # RZ(alpha_i)门作用在第i位量子比特
encoder = encoder.no_grad()
encoder = encoder.as_encoder()# Encoder作为整个量子神经网络的第一层，不用对编码线路中的梯度求导数，因此加入no_grad()der
# encoder.svg()

In [4]:
ansatz = HardwareEfficientAnsatz(8, single_rot_gate_seq=[RY], entangle_gate=X, depth=2).circuit   
# ansatz.summary()                                    # 总结Ansatz

In [5]:
from mindspore.nn import Adam, TrainOneStepCell    

In [6]:
ansatz = HardwareEfficientAnsatz(nqbits, single_rot_gate_seq=[RY], entangle_gate=X, depth=2).circuit     # 通过
hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
ansatz = ansatz.as_ansatz()
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(2)                                                     # 设置生成随机数的种子
circuit = encoder+ ansatz.as_ansatz()         
sim = Simulator('mqvector', n_qubits=nqbits)
grad_ops = sim.get_expectation_with_grad(hams,
                                         circuit,
                                         parallel_worker=5)
QuantumNet = MQLayer(grad_ops)          # 搭建量子神经网络
ops = MQOps(grad_ops)

In [ ]:
ops(ms.Tensor(X_train[0:5]),ms.Tensor(np.random.randn(24)))

In [ ]:
list(ops.get_parameters())

In [ ]:
QuantumNet.trainable_params()[0]

In [ ]:
# nnp = ms.Tensor(np.random.randn(24))
np.random.seed(1)
nnp = ms.Tensor(np.random.randn(24))

# nnp = np.random.normal(loc=0.0, scale=0.02, size=(3,8)).astype(np.float64)
# nnp = ms.Tensor(np.random.randn(24))
nnp

In [ ]:
loss_fn = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') # 定义损失函数    

def forward_fn(encode_p,ansatz_p,y_label):
    eval_obserables = ops(encode_p,ansatz_p)
    loss = loss_fn(eval_obserables, y_label)
    return loss

index = 100
grad_fn = ms.value_and_grad(fn=forward_fn,grad_position=None,weights=nnp)
value,grad = grad_fn(ms.Tensor(X_train[0:index]),nnp,ms.Tensor(y_train[0:index]))
print(value,grad)

In [ ]:
nnp

In [66]:
QuantumNet = MQLayer(grad_ops)          # 搭建量子神经网络

In [ ]:
QuantumNet

In [ ]:
grad_ops(X_train[0],np.random.randn(24))

In [ ]:
QuantumNet.trainable_params()

In [ ]:

QuantumNet(ms.Tensor(X_train[0:5]))

In [45]:
loss_fn = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') # 定义损失函数    
def forward_fn(x, y):
    z = QuantumNet(x)
    #print(z)
    loss = loss_fn(z, y)
    return loss

In [46]:
grad_fn = ms.value_and_grad(forward_fn, None, weights=QuantumNet.trainable_params())

In [47]:
value,grad = grad_fn(ms.Tensor(X_train[0:5]), ms.Tensor(y_train[0:5]))

In [ ]:
value

In [ ]:
grad

In [24]:
opti = Adam(QuantumNet.trainable_params(), learning_rate=0.5)     # 需要优化的是Quantumnet中可训练的参数，学习率设为0.5
net = TrainOneStepCell(QuantumNet, opti,return_grad=True)

In [ ]:
from mindspore import Tensor
Tensor(X_train[0])

In [122]:
opti = Adam(QuantumNet.trainable_params(), learning_rate=0.5) 
net = TrainOneStepCell(ops, opti,return_grad=True)

In [ ]:
net.trainable_params()

In [ ]:
from mindspore import Tensor
net(Tensor(X_train[0]),Tensor(y_train[0]))

In [ ]:
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集



loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')            # 通过SoftmaxCrossEntropyWithLogits定义损失函数，sparse=True表示指定标签使用稀疏格式，reduction='mean'表示损失函数的降维方法为求平均值
opti = Adam(QuantumNet.trainable_params(), learning_rate=0.01)                  # 通过Adam优化器优化Ansatz中的参数，需要优化的是Quantumnet中可训练的参数，学习率设为0.1

model = Model(QuantumNet, loss, opti, metrics={'Acc': Accuracy()})             # 建立模型：将MindSpore Quantum构建的量子机器学习层和MindSpore的算子组合，构成一张更大的机器学习网络


train_loader = NumpySlicesDataset({'features': X_train, 'labels': y_train}, shuffle=False).batch(20) # 通过NumpySlicesDataset创建训练样本的数据集，shuffle=False表示不打乱数据，batch(5)表示训练集每批次样本点有5个
test_loader = NumpySlicesDataset({'features': X_test, 'labels': y_test},shuffle=False).batch(20)                   # 通过NumpySlicesDataset创建测试样本的数据集，batch(5)表示测试集每批次样本点有5个


class StepAcc(ms.Callback):                                                      # 定义一个关于每一步准确率的回调函数
    def __init__(self, model, test_loader):
        self.model = model
        self.test_loader = test_loader
        self.acc = []

    def on_train_step_end(self, run_context):
        
        self.acc.append(self.model.eval(self.test_loader, dataset_sink_mode=False)['Acc'])
        # print(f'ACC = {self.acc[-1]}')


monitor = LossMonitor(20)                                                       # 监控训练中的损失，每16步打印一次损失值

acc = StepAcc(model, test_loader)                                               # 使用建立的模型和测试样本计算预测的准确率

model.train(20, train_loader, callbacks=[monitor, acc], dataset_sink_mode=False)# 将上述建立好的模型训练20次

In [ ]:
plt.plot(acc.acc)
plt.title('Statistics of accuracy', fontsize=20)
plt.xlabel('Steps', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)

In [ ]:
from Test_tool import  Test_ansatz

In [ ]:
accz = Test_ansatz()


In [ ]:
accz[-1]